In [ ]:
!gdown --id 1UhIF3TqmBxWGJIj6vNsYUx0iLiNnOd0q

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1UhIF3TqmBxWGJIj6vNsYUx0iLiNnOd0q
To: /content/train.csv
100% 18.3M/18.3M [00:00<00:00, 93.2MB/s]


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Dropout, Embedding, Conv1D, MaxPooling1D, LSTM, Flatten
from tensorflow.keras.models import Sequential
import pickle


In [ ]:
train_df = pd.read_csv('train.csv')

In [ ]:
train_df.head()

,text,dialect
0,قولتله اصلي عياده الدكتور اهيه فوقينا اتكسف...,EG
1,متحاولش تبقي مبهور بحد مشفتوش وهو بياكل وهو صا...,EG
2,كنك متردش عليا ؟ والله كنت ماسك النقال وشفت ...,LY
3,بتزيح او بقعد بحضنك,LB
4,مفيش حد كامل من كل شء مفيش حد مش بيغلط مغيش حد...,EG


In [ ]:
texts = train_df['text'].values.astype(str)
labels = train_df['dialect'].values

In [ ]:
train_df['dialect'].unique()


array(['EG', 'LY', 'LB', 'SD', 'MA'], dtype=object)

In [ ]:

# Map string labels to integer labels
label_map = {'EG': 0, 'LB': 1, 'MA': 2, 'SD': 3, 'LY': 4}
labels = [label_map[label] for label in labels]


In [ ]:
# Tokenize text data
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=5000)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))


Found 211859 unique tokens.


In [ ]:
# Save the tokenizer
tokenizer_path = 'tokenizer.pkl'
with open(tokenizer_path, 'wb') as file:
    pickle.dump(tokenizer, file)

In [ ]:
# Pad sequences to same length
max_sequence_length = max([len(seq) for seq in sequences])
data = pad_sequences(sequences, maxlen=max_sequence_length)
labels = np.asarray(labels)

In [ ]:
# Split data into train and test sets
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
num_validation_samples = int(0.2 * data.shape[0])

x_train = data[:-num_validation_samples]
y_train = labels[:-num_validation_samples]
x_test = data[-num_validation_samples:]
y_test = labels[-num_validation_samples:]


In [ ]:
# Convert labels to one-hot encoded vectors
y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)

In [ ]:
# Build model
model = Sequential()
model.add(Embedding(5000, 128, input_length=max_sequence_length))
model.add(Conv1D(filters=64, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(5, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
model.fit(x_train, y_train,
          validation_data=(x_test, y_test),
          epochs=3, batch_size=128)

Epoch 1/3
831/831 [==============================] - 246s 296ms/step - loss: 0.5126 - accuracy: 0.8143 - val_loss: 0.5699 - val_accuracy: 0.7923
Epoch 2/3
831/831 [==============================] - 232s 280ms/step - loss: 0.4623 - accuracy: 0.8320 - val_loss: 0.5817 - val_accuracy: 0.7910
Epoch 3/3
831/831 [==============================] - 240s 289ms/step - loss: 0.4167 - accuracy: 0.8500 - val_loss: 0.5983 - val_accuracy: 0.7869


In [ ]:
# Evaluate model
loss, accuracy = model.evaluate(x_test, y_test, verbose=0)
print('Accuracy: %f' % (accuracy*100))

Accuracy: 78.688091


In [ ]:
model.save('DL3.h5')